In [70]:
import pandas as pd
import requests
from datetime import datetime
from sqlalchemy import create_engine
import os
uri = os.environ.get('URI')


url = 'https://api.blockchain.info/charts/transactions-per-second?timespan=all&sampled=false&metadata=false&cors=true&format=json'

resp = requests.get(url)


In [71]:
data = pd.DataFrame(resp.json()["values"])

In [72]:
# from timestamp to date
data['x'] = [datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in data['x']]
data['x'] = pd.to_datetime(data['x'])

In [73]:
data.head()

,x,y
0,2016-06-11 00:00:00,1.983333
1,2016-06-11 00:15:00,1.966667
2,2016-06-11 00:30:00,1.900000
3,2016-06-11 00:45:00,2.283333
4,2016-06-11 01:00:00,2.083333


In [74]:
data.columns = ["date", "transactions"]

In [75]:
data.head()

,date,transactions
0,2016-06-11 00:00:00,1.983333
1,2016-06-11 00:15:00,1.966667
2,2016-06-11 00:30:00,1.900000
3,2016-06-11 00:45:00,2.283333
4,2016-06-11 01:00:00,2.083333


In [76]:
data['date'] = data['date'].dt.round('H')
grouped_data = data.groupby('date').sum().reset_index()


In [77]:
grouped_data.head()

,date,transactions
0,2016-06-11 00:00:00,5.85
1,2016-06-11 01:00:00,6.05
2,2016-06-11 02:00:00,9.05
3,2016-06-11 03:00:00,6.05
4,2016-06-11 04:00:00,12.65


In [78]:
# switched from DataFrame to pandas.series (which is just one/each column of dataframe)
grouped_data = grouped_data.set_index('date')
grouped_data = grouped_data['transactions']
grouped_data.head()

date
2016-06-11 00:00:00     5.85
2016-06-11 01:00:00     6.05
2016-06-11 02:00:00     9.05
2016-06-11 03:00:00     6.05
2016-06-11 04:00:00    12.65
Name: transactions, dtype: float64

In [79]:
type(grouped_data)

pandas.core.series.Series

In [80]:
steps = 36
n_datos_entrenar = 200

train_data = grouped_data[ -n_datos_entrenar:-steps]
test_data  = grouped_data[-steps:]

In [95]:
#from utils import create_predictors
from skforecast.model_selection import grid_search_forecaster
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.ensemble import RandomForestRegressor

# imports for data reading
import requests
from datetime import datetime

# imports for Neptune
import os
from dotenv import load_dotenv
import neptune

In [88]:
# ???? predictors generation not included in tutorial
import numpy as np
def create_predictors(y):
    """
    Create first 10 lags of a time series.
    Calculate moving average with window 20.
    """
    
    lags = y[-1:-11:-1]     # window size needed = 10
    mean = np.mean(y[-20:]) # window size needed = 20
    predictors = np.hstack([lags, mean])
    
    return predictors


# fixed predictors creating

# def create_predictors(y):
#     """
#     Create first 10 lags of a time series.
#     Calculate moving average with window 20.
#     """

#     lags = y.iloc[-1:-11:-1]     # window size needed = 10
#     mean = np.mean(y.iloc[-20:]) # window size needed = 20
#     predictors = np.hstack([lags, mean])

#     return predictors

In [97]:
# Define forecaster
forecaster_rf = ForecasterAutoreg(
                    regressor      = RandomForestRegressor(random_state=123),
                    #fun_predictors = create_predictors,
                    lags    = 20
                )

In [90]:
param_grid = { 'n_estimators': [100, 500], 'max_depth': [3, 5, 10] }

In [91]:
train_data.head()

date
2024-04-02 05:00:00    15.800000
2024-04-02 06:00:00    17.283333
2024-04-02 07:00:00     9.100000
2024-04-02 08:00:00    19.366667
2024-04-02 09:00:00    13.216667
Name: transactions, dtype: float64

In [94]:
create_predictors(train_data)

array([16.41666667,  9.8       , 16.73333333,  8.81666667, 18.8       ,
       11.88333333, 24.96666667, 17.15      , 39.63333333, 35.78333333,
       21.92833333])

In [92]:
grid_results = grid_search_forecaster(
                        forecaster  = forecaster_rf,
                        y           = train_data,
                        param_grid  = param_grid,
                        steps       = 10,
                        metric      = 'mean_squared_error',
                        initial_train_size    = int(len(train_data)*0.5),
                        allow_incomplete_fold = True,
                        return_best = True, # return only ONE best grid
                        verbose     = False
                    )

Number of models compared: 6.


lags grid:   0%|          | 0/1 [00:00<?, ?it/s]/home/michal/documents/MLOps-pipeline-github-actions/venv/lib/python3.10/site-packages/skforecast/utils/utils.py:805: UserWarning: `y` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
/home/michal/documents/MLOps-pipeline-github-actions/venv/lib/python3.10/site-packages/skforecast/utils/utils.py:867: UserWarning: `last_window` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
/home/michal/documents/MLOps-pipeline-github-actions/venv/lib/python3.10/site-packages/skforecast/utils/utils.py:867: UserWarning: `last_window` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
/home/michal/documents/MLOps-pipeline-github-actions/venv/lib/python3.10/site-packages/skforecast/utils/utils.py:867: UserWarning: `last_window` has DatetimeIndex index but no frequency. Index is over

KeyError: "None of [Index([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 20, 21, 22, 23, 24, 25, 26, 27,\n       28, 29, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 20, 21, 22, 23, 24, 25,\n       26, 27, 28, 29, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 20, 21, 22, 23,\n       24, 25, 26, 27, 28, 29, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 20, 21,\n       22, 23, 24, 25, 26, 27, 28, 29, 20, 21],\n      dtype='int64')] are in the [index]"